# https://www.learnpytorch.io/

In [ ]:
import torch

In [ ]:
torch.__version__

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.current_device()

In [ ]:
cuda0 = torch.device('cuda:0')

In [ ]:
torch.cuda.device_count()

# Tensors

In [ ]:
t1 = torch.rand(2000, 100)
t2 = torch.rand(100, 2000)

In [ ]:
%%timeit -n 100
t1 @ t2

In [ ]:
t1_ongpu = t1.to(cuda0)
t2_ongpu = t2.to(cuda0)

In [ ]:
%%timeit -n 100
t1_ongpu @ t2_ongpu

In [ ]:
t2_ongpu.device

In [ ]:
t = t2_ongpu.cpu()

In [ ]:
type(t)

In [ ]:
np_t1 = t.numpy()

In [ ]:
np_t1

# Learn Linear Regression

### Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
mu = 0
sigma = 10
nois = np.random.normal(mu, sigma, 10)

In [ ]:
x = np.arange(1, 100, 1)
b = 3
# y_model = np.power(x, 2) + b

y_model = 2.4 * x + 7.5  * np.sin(0.4 * x) + b
data = {'x': x,
        'y_m': y_model,
        'y': y_model + np.random.normal(mu, sigma, y_model.size)
        #'y': y_model +  np.power(x, 1.25) * np.random.normal(mu, sigma, y_model.size)
       }

In [ ]:
# fig, ax = plt.subplots()

In [ ]:
plt.plot(x, data['y_m'])
plt.scatter(x, data['y'], s=15)
plt.show()

### Torch Model

In [ ]:
from torch import nn
from torch import from_numpy

In [ ]:
train_split = int(0.7 * len(data['x'])) # 80% of data used for training set, 20% for testing 
X_train, y_train = data['x'][:train_split], data['y'][:train_split]
X_test, y_test = data['x'][train_split:], data['y'][train_split:]

### Print Learning data test/trian split

In [ ]:
def plot_predictions(train_data=X_train, 
                     train_labels=y_train, 
                     test_data=X_test, 
                     test_labels=y_test, 
                     predictions=None):
    """
    Plots training data, test data and compares predictions.
    """
    plt.figure(figsize=(10, 7))

    # Plot training data in blue
    plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")

    # Plot test data in green
    plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

    if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
        plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

    # Show the legend
    plt.legend(prop={"size": 14});

In [ ]:
plot_predictions()

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__() 
        self.weights = nn.Parameter(torch.randn(1, # <- start with random weights (this will get adjusted as the model learns)
                                                dtype=torch.float), # <- PyTorch loves float32 by default
                                    requires_grad=True) # <- can we update this value with gradient descent?)

        self.bias = nn.Parameter(torch.randn(1, # <- start with random bias (this will get adjusted as the model learns)
                                            dtype=torch.float), # <- PyTorch loves float32 by default
                                requires_grad=True) # <- can we update this value with gradient descent?))

    # Forward defines the computation in the model
    def reset_weights(self):
        self.weights = nn.Parameter(torch.randn(1,
                                                dtype=torch.float),
                                    requires_grad=True)

        self.bias = nn.Parameter(torch.randn(1,
                                            dtype=torch.float),
                                requires_grad=True)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias

In [ ]:
model_0 = LinearRegressionModel()

In [ ]:
list(model_0.parameters())

In [ ]:
model_0.state_dict()

In [ ]:
with torch.inference_mode(): 
    y_preds = model_0(X_test)

In [ ]:
y_preds

In [ ]:
y_preds.detach().numpy()

In [ ]:
X_test

In [ ]:
plot_predictions(predictions=y_preds.detach().numpy())

In [ ]:
torch.device

In [ ]:
# model_0_gpu = model_0.to(cuda0)

In [ ]:
#print(model_0_gpu.state_dict())

In [ ]:
#print(model_0.state_dict())

In [ ]:
#print(model_0.state_dict())

In [ ]:
#loss_fn = nn.L1Loss() # MAE loss is same as L1Loss

# Create the optimizer
#optimizer = torch.optim.SGD(params=model_0_gpu.parameters(), # parameters of target model to optimize
#                            lr=0.01) 

### Prepare data to learning

In [ ]:
# Set the number of epochs (how many times the model will pass over the training data)
epochs = 100
tX_train=from_numpy(X_train)
ty_train=from_numpy(y_train)

tX_test=from_numpy(X_test)
ty_test=from_numpy(y_test)
# Create empty loss lists to track values
train_loss_values = []
test_loss_values = []
epoch_count = []

In [ ]:
epochs = 100
epochs = 100
# Create empty loss lists to track values
train_loss_values = []
test_loss_values = []
epoch_count = []
#torch.manual_seed(42)
def train(model=model_0,
          X_test=tX_test,
          X_train=tX_train,
          y_train=ty_train,
          y_test=ty_test,
          epochs=epochs,
          train_loss_values=train_loss_values,
          test_loss_values=test_loss_values,
          epoch_count=epoch_count
         ):
    loss_fn = nn.L1Loss()
    optimizer = torch.optim.SGD(params=model_0.parameters(), # parameters of target model to optimize
                            lr=0.01)
    for epoch in range(epochs):
        ### Training

        # Put model in training mode (this is the default state of a model)
        model.train()

        # 1. Forward pass on train data using the forward() method inside 
        y_pred = model(X_train)
        # print(y_pred)

        # 2. Calculate the loss (how different are our models predictions to the ground truth)
        loss = loss_fn(y_pred, y_train)

        # 3. Zero grad of the optimizer
        optimizer.zero_grad()

        # 4. Loss backwards
        loss.backward()

        # 5. Progress the optimizer
        optimizer.step()

        ### Testing

        # Put the model in evaluation mode
        model.eval()

        with torch.inference_mode():
          # 1. Forward pass on test data
          test_pred = model(X_test)

          # 2. Caculate loss on test data
          test_loss = loss_fn(test_pred, y_test.type(torch.float)) # predictions come in torch.float datatype, so comparisons need to be done with tensors of the same type

          # Print out what's happening
          if epoch % 10 == 0:
                #epoch_count.append(epoch)
                #curr_loss = loss.detach()
                #curr_test_loss = test_loss.detach()
                #if (curr_loss.device.type == 'cuda'):
                #    curr_loss = curr_loss.cpu()
                #    curr_test_loss = curr_test_loss.cpu()
                #train_loss_values.append(curr_loss.numpy())
                #test_loss_values.append(curr_test_loss.numpy())
                print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss} ")

In [ ]:
model_0.reset_weights()
print(model_0.state_dict())
%time train()
print(model_0.state_dict())

In [ ]:
plt.plot(epoch_count, train_loss_values, label="Train loss")
plt.plot(epoch_count, test_loss_values, label="Test loss")
plt.title("Training and test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend();

In [ ]:
print(model_0.state_dict())
with torch.inference_mode(): 
    y_preds = model_0(X_test)
plot_predictions(predictions=y_preds.detach().numpy())

In [ ]:
model_0.reset_weights()
model_0.to(cuda0)
print(model_0.state_dict())
params = {
    'X_test':tX_test.to(cuda0),
    'X_train':tX_train.to(cuda0),
    'y_train':ty_train.to(cuda0),
    'y_test':ty_test.to(cuda0)
}
%time train(**params)
print(model_0.state_dict())

In [ ]:
for p in model_0.parameters():
    print (p)

In [ ]:
tgpuX_test = tX_test.to(cuda0)
tgpuX_test.device.type

In [ ]:
tgpuX_test.cpu().device.type

## Model Saving

In [ ]:
torch.save(obj=model_0.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f='model_0')

In [ ]:
print(model_0.state_dict())

In [ ]:

# Instantiate a new instance of our model (this will be instantiated with random weights)
loaded_model_0 = LinearRegressionModel()

# Load the state_dict of our saved model (this will update the new instance of our model with trained weights)
loaded_model_0.load_state_dict(torch.load(f='model_0'))

In [ ]:
print(loaded_model_0.state_dict())

In [ ]:
with torch.inference_mode():
    loaded_model_preds = loaded_model_0(tX_test)

In [ ]:
loaded_model_preds[:5]